We assume that demand is deterministic and constant with a rate of λ units per year.
Stockouts are not allowed—we must always order enough so that demand can be met.
Since demand is deterministic, this is a plausible assumption. The lead time is 0—orders
are received instantaneously. There is a fixed cost K per order, a purchase cost c per unit
ordered, and an inventory holding cost h per unit per year. There is no stockout penalty
since stockouts are not allowed.

Any optimal solution for the EOQ model has two important properties:
    
• Zeroinventory
ordering (ZIO) property. Since the lead time is 0, it never makes
sense to place an order when there is a positive amount of inventory on hand—we
only place an order when the inventory level is 0.
    
• Constant order sizes. If Q is the optimal order size at time 0, it will also be the
optimal order size every other time we place an order since the system looks the
same every time the inventory level hits 0. Therefore, the order size is the same every
time an order is placed.
    


In [1]:
# Practice 0
function EOQ_cost(lambda, K, h, Q)
    cost = (K .* lambda ./ Q) + (h .* Q ./ 2)
    return cost
end

EOQ_cost (generic function with 1 method)

In [2]:
lambda = 1300
K = 8
h = 0.225
Q = 300:10:350
EOQ_cost(lambda, K, h, Q)

6-element Vector{Float64}:
 68.41666666666666
 68.42338709677419
 68.5
 68.64015151515152
 68.83823529411765
 69.08928571428572

should emphasize that scalar and vector have different operants (. and w/o .)
you will get error if you don't put . before your operants
#TWO EXPRESSION

In [3]:
# Practice 1 and Practice 2
function EOQ_cost_and_quantity(lambda, K, h)
    quantity = sqrt.(2 .* K .* lambda ./ h)
    cost = h .* quantity
    return quantity, cost
end

EOQ_cost_and_quantity (generic function with 1 method)

In [4]:
EOQ_cost_and_quantity([1000,2000], [25,30], [3,5])

([129.09944487358058, 154.91933384829667], [387.29833462074174, 774.5966692414834])

In [5]:
function EOQ_cost_and_quantity2(lambda, K, h)
    quantity = sqrt(2 * K * lambda / h)
    cost = h * quantity
    return quantity, cost
end

EOQ_cost_and_quantity2 (generic function with 1 method)

In [6]:
EOQ_cost_and_quantity2.([1000,2000], [25,30], [3,5])

2-element Vector{Tuple{Float64, Float64}}:
 (129.09944487358058, 387.29833462074174)
 (154.91933384829667, 774.5966692414834)

#Practice 3

INPUT ARGUMENTS: <br>
   lambda  = demand rate [scalar or vector] <br>
   K       = fixed cost [scalar or vector] <br>
   i       = holding rate as % [scalar or vector] <br>
   b       = breakpoints b_1,...,b_n [n-vector or n x m matrix] (m = # of instances) <br>
   c       = purchase prices c_0,...,c_n [(n+1)-vector or (n+1) x m matrix] <br>
   n       = number of breakpoints [scalar or vector] <br>
<br>
OUTPUT ARGUMENTS: <br>
   Q       = optimal order quantity [scalar or vector] <br>
   g       = cost g(Q) [scalar or vector] <br>

In [7]:
function EOQ_AllUnitDiscount(lambda, K, i, b, c, n)
    m = length(lambda)
    
    # store results for best quantity and best cost
    Q = zeros(1, m)
    g = zeros(1, m)
    
    b_with0 = [zeros(1, m); b]
    
    for t in 1:m
        Qstar = sqrt.(2 .* K[t] .* lambda[t] ./ (i[t] .* c[:,t]))
        jstar = 0
        bestg = 0
        bestQ = 0
        
        for j in (n[t]+1):-1:1
            if j == n[t]+1
                realizable = (Qstar[j] >= b_with0[j,t])
            else
                realizable = ( (Qstar[j] >= b_with0[j,t]) & (Qstar[j] < b_with0[j+1,t]) )
            end
            
            if realizable 
                tempg = c[j,t] * lambda[t] + K[t] * lambda[t] / Qstar[j] + i[t] * c[j,t] * Qstar[j] / 2;
                        
                bestg = tempg;
                bestQ = Qstar[j];
                
                jstar = j;
                break;
            end
        end
        
        for j in (jstar+1):(n[t]+1)
            tempg = c[j,t] * lambda[t] + K[t] * lambda[t] / b_with0[j,t] + i[t] * c[j,t] * b_with0[j,t] / 2
            if tempg < bestg
                bestg = tempg
                bestQ = b_with0[j,t]
            end
        end

        Q[t] = bestQ;
        g[t] = bestg;
        
    end

    return Q, g
end

EOQ_AllUnitDiscount (generic function with 1 method)

In [8]:
lambda = [380 1125 150 100]
K = [80 12 100 15]
i = [0.25 0.30 0.16 0.22]
n = [3 2 3 2]
b = [100 200 300; 400 500 0; 115 125 135; 60 250 0]'
c = [10 9 8 7.9; 0.6 0.4 0.3 0; 15 13.8 12.6 12.2; 4 3.2 3 0]'

EOQ_AllUnitDiscount(lambda, K, i, b, c, n)

([200.0 547.7225575051662 135.0 65.27912098338669], [3392.0 386.79503017546494 2072.871111111111 365.95650117230423])